# Home Credit Default Risk

## Abstract:
this notebook explores the data related to home credit default risk competition in Kaggle. The data will be combined toegther and then feature exploration analysis is done. Finaly a prediction has been done and prepared to be sumbited to Kaggle

## Import Libraries

In [2]:
import pandas as pd
import numpy as np

## load data
there are 10 files:
* 8 related to data that we need to build the model
* 1 related to data where we have to use our model to predict the labels
* 1 is an example shows how we should submit

Load file 1: application_train.csv.zip

In [5]:
df_train = pd.read_csv('application_train.csv.zip',compression = 'gzip')

FileNotFoundError: [Errno 2] No such file or directory: 'application_train.csv.zip'